In [18]:
%matplotlib notebook
from sklearn import model_selection, preprocessing, metrics
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import GridSearchCV
import seaborn as sn
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, accuracy_score

Загружаем данные

In [2]:
train = pd.read_csv('/home/elavelina/Documents/DSML_2018-master/HW/5/train.csv')
macro = pd.read_csv('/home/elavelina/Documents/DSML_2018-master/HW/5/macro.csv')
test = pd.read_csv('/home/elavelina/Documents/DSML_2018-master/HW/5/train.csv')
train_joined = pd.merge(macro, train, on='timestamp')
test_joined = pd.merge(macro, test, on='timestamp')

In [3]:
train_joined = train_joined.fillna(-1)
train_joined = train_joined.replace('yes', 1)
train_joined = train_joined.replace('no', 0)
train_joined.head()

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,2011-08-20,109.31,14313.7,3.3,354.0,420.7,86.721,15.459,10.1,29.0048,...,9,4,0,13,22,1,0,52,4,5850000
1,2011-08-23,109.31,14313.7,3.3,354.0,420.7,86.721,15.459,10.1,28.9525,...,15,3,0,15,29,1,10,66,14,6000000
2,2011-08-27,109.31,14313.7,3.3,354.0,420.7,86.721,15.459,10.1,28.8082,...,10,3,0,11,27,0,4,67,10,5700000
3,2011-09-01,111.29,14313.7,3.3,353.2,434.4,86.721,15.386,10.1,28.9655,...,11,2,1,4,4,0,0,26,3,13100000
4,2011-09-05,111.29,14313.7,3.3,353.2,434.4,86.721,15.386,10.1,29.4625,...,319,108,17,135,236,2,91,195,14,16331452


In [4]:
test_joined = test_joined.fillna(-1)
test_joined = test_joined.replace('yes', 1)
test_joined = test_joined.replace('no', 0)
test_joined.head()

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,2011-08-20,109.31,14313.7,3.3,354.0,420.7,86.721,15.459,10.1,29.0048,...,9,4,0,13,22,1,0,52,4,5850000
1,2011-08-23,109.31,14313.7,3.3,354.0,420.7,86.721,15.459,10.1,28.9525,...,15,3,0,15,29,1,10,66,14,6000000
2,2011-08-27,109.31,14313.7,3.3,354.0,420.7,86.721,15.459,10.1,28.8082,...,10,3,0,11,27,0,4,67,10,5700000
3,2011-09-01,111.29,14313.7,3.3,353.2,434.4,86.721,15.386,10.1,28.9655,...,11,2,1,4,4,0,0,26,3,13100000
4,2011-09-05,111.29,14313.7,3.3,353.2,434.4,86.721,15.386,10.1,29.4625,...,319,108,17,135,236,2,91,195,14,16331452


In [5]:
img = sn.heatmap(train.corr(), cmap="YlGnBu", square=True, xticklabels=False, yticklabels=False).get_figure()


<IPython.core.display.Javascript object>

<div id='e4a6434a-355b-4c11-95af-5e6d77bdcae6'></div>

In [6]:
img.savefig('/home/elavelina/Documents/DSML_2018-master/HW/5/task5_img1.png', bbox_inches='tight')

In [8]:
corr_matrix = train_joined.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
corr_features = [column for column in upper.columns if any(upper[column] > 0.95)]
print(corr_features)

['ppi', 'usdrub', 'eurrub', 'brent', 'gdp_annual', 'grp_growth', 'real_dispos_income_per_cap_growth', 'salary', 'fixed_basket', 'retail_trade_turnover', 'retail_trade_turnover_per_cap', 'retail_trade_turnover_growth', 'labor_force', 'employment', 'invest_fixed_capital_per_cap', 'invest_fixed_assets', 'profitable_enterpr_share', 'unprofitable_enterpr_share', 'share_own_revenues', 'fin_res_per_cap', 'marriages_per_1000_cap', 'divorce_rate', 'construction_value', 'invest_fixed_assets_phys', 'pop_total_inc', 'housing_fund_sqm', 'lodging_sqm_per_cap', 'water_pipes_share', 'baths_share', 'sewerage_share', 'gas_share', 'hot_water_share', 'electric_stove_share', 'heating_share', 'old_house_share', 'average_life_exp', 'infant_mortarity_per_1000_cap', 'perinatal_mort_per_1000_cap', 'incidence_population', 'load_of_teachers_preschool_per_teacher', 'provision_doctors', 'provision_nurse', 'power_clinics', 'hospital_beds_available_per_cap', 'hospital_bed_occupancy_per_year', 'turnover_catering_per_c

In [13]:
corr_train = train_joined[['ppi', 'usdrub', 'eurrub', 'brent', 'gdp_annual', 'grp_growth', 'real_dispos_income_per_cap_growth', 'salary', 'fixed_basket', 'retail_trade_turnover', 'retail_trade_turnover_per_cap', 'retail_trade_turnover_growth', 'labor_force', 'employment', 'invest_fixed_capital_per_cap', 'invest_fixed_assets', 'profitable_enterpr_share', 'unprofitable_enterpr_share', 'share_own_revenues', 'fin_res_per_cap', 'marriages_per_1000_cap', 'divorce_rate', 'construction_value', 'invest_fixed_assets_phys', 'pop_total_inc', 'housing_fund_sqm', 'lodging_sqm_per_cap', 'water_pipes_share', 'baths_share', 'sewerage_share', 'gas_share', 'hot_water_share', 'electric_stove_share', 'heating_share', 'old_house_share', 'average_life_exp', 'infant_mortarity_per_1000_cap', 'perinatal_mort_per_1000_cap', 'incidence_population', 'load_of_teachers_preschool_per_teacher', 'provision_doctors', 'provision_nurse', 'power_clinics', 'hospital_beds_available_per_cap', 'hospital_bed_occupancy_per_year', 'turnover_catering_per_cap', 'theaters_viewers_per_1000_cap', 'seats_theather_rfmin_per_100000_cap', 'museum_visitis_per_100_cap', 'bandwidth_sports', 'population_reg_sports_share', 'students_reg_sports_share', 'apartment_build', 'apartment_fund_sqm', 'id', 'children_preschool', 'children_school', 'school_quota', 'school_education_centers_raion', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_builddate_info', 'metro_min_walk', 'metro_km_walk', 'railroad_station_walk_min', 'railroad_station_avto_km', 'public_transport_station_min_walk', 'sadovoe_km', 'bulvar_ring_km', 'kremlin_km', 'zd_vokzaly_avto_km', 'preschool_km', 'cafe_sum_500_max_price_avg', 'cafe_avg_price_500', 'cafe_count_500_price_500', 'cafe_count_500_price_1500', 'cafe_count_500_price_2500', 'cafe_count_1000', 'cafe_sum_1000_max_price_avg', 'cafe_avg_price_1000', 'cafe_count_1000_na_price', 'cafe_count_1000_price_500', 'cafe_count_1000_price_1000', 'cafe_count_1000_price_1500', 'cafe_count_1000_price_2500', 'cafe_count_1000_price_4000', 'church_count_1000', 'office_count_1500', 'cafe_count_1500', 'cafe_sum_1500_max_price_avg', 'cafe_avg_price_1500', 'cafe_count_1500_na_price', 'cafe_count_1500_price_500', 'cafe_count_1500_price_1000', 'cafe_count_1500_price_1500', 'cafe_count_1500_price_2500', 'cafe_count_1500_price_4000', 'big_church_count_1500', 'church_count_1500', 'leisure_count_1500', 'office_count_2000', 'cafe_count_2000', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'cafe_count_2000_na_price', 'cafe_count_2000_price_500', 'cafe_count_2000_price_1000', 'cafe_count_2000_price_1500', 'cafe_count_2000_price_2500', 'cafe_count_2000_price_4000', 'cafe_count_2000_price_high', 'big_church_count_2000', 'church_count_2000', 'leisure_count_2000', 'office_count_3000', 'office_sqm_3000', 'cafe_count_3000', 'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'cafe_count_3000_na_price', 'cafe_count_3000_price_500', 'cafe_count_3000_price_1000', 'cafe_count_3000_price_1500', 'cafe_count_3000_price_2500', 'cafe_count_3000_price_4000', 'cafe_count_3000_price_high', 'big_church_count_3000', 'church_count_3000', 'leisure_count_3000', 'office_count_5000', 'office_sqm_5000', 'cafe_count_5000', 'cafe_sum_5000_max_price_avg', 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high', 'big_church_count_5000', 'church_count_5000', 'leisure_count_5000', 'sport_count_5000']]

In [15]:
X_train = corr_train.drop(["id"], axis=1)
Y_train = train_joined['price_doc']

In [16]:
corr_test = test_joined[['ppi', 'usdrub', 'eurrub', 'brent', 'gdp_annual', 'grp_growth', 'real_dispos_income_per_cap_growth', 'salary', 'fixed_basket', 'retail_trade_turnover', 'retail_trade_turnover_per_cap', 'retail_trade_turnover_growth', 'labor_force', 'employment', 'invest_fixed_capital_per_cap', 'invest_fixed_assets', 'profitable_enterpr_share', 'unprofitable_enterpr_share', 'share_own_revenues', 'fin_res_per_cap', 'marriages_per_1000_cap', 'divorce_rate', 'construction_value', 'invest_fixed_assets_phys', 'pop_total_inc', 'housing_fund_sqm', 'lodging_sqm_per_cap', 'water_pipes_share', 'baths_share', 'sewerage_share', 'gas_share', 'hot_water_share', 'electric_stove_share', 'heating_share', 'old_house_share', 'average_life_exp', 'infant_mortarity_per_1000_cap', 'perinatal_mort_per_1000_cap', 'incidence_population', 'load_of_teachers_preschool_per_teacher', 'provision_doctors', 'provision_nurse', 'power_clinics', 'hospital_beds_available_per_cap', 'hospital_bed_occupancy_per_year', 'turnover_catering_per_cap', 'theaters_viewers_per_1000_cap', 'seats_theather_rfmin_per_100000_cap', 'museum_visitis_per_100_cap', 'bandwidth_sports', 'population_reg_sports_share', 'students_reg_sports_share', 'apartment_build', 'apartment_fund_sqm', 'id', 'children_preschool', 'children_school', 'school_quota', 'school_education_centers_raion', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_builddate_info', 'metro_min_walk', 'metro_km_walk', 'railroad_station_walk_min', 'railroad_station_avto_km', 'public_transport_station_min_walk', 'sadovoe_km', 'bulvar_ring_km', 'kremlin_km', 'zd_vokzaly_avto_km', 'preschool_km', 'cafe_sum_500_max_price_avg', 'cafe_avg_price_500', 'cafe_count_500_price_500', 'cafe_count_500_price_1500', 'cafe_count_500_price_2500', 'cafe_count_1000', 'cafe_sum_1000_max_price_avg', 'cafe_avg_price_1000', 'cafe_count_1000_na_price', 'cafe_count_1000_price_500', 'cafe_count_1000_price_1000', 'cafe_count_1000_price_1500', 'cafe_count_1000_price_2500', 'cafe_count_1000_price_4000', 'church_count_1000', 'office_count_1500', 'cafe_count_1500', 'cafe_sum_1500_max_price_avg', 'cafe_avg_price_1500', 'cafe_count_1500_na_price', 'cafe_count_1500_price_500', 'cafe_count_1500_price_1000', 'cafe_count_1500_price_1500', 'cafe_count_1500_price_2500', 'cafe_count_1500_price_4000', 'big_church_count_1500', 'church_count_1500', 'leisure_count_1500', 'office_count_2000', 'cafe_count_2000', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'cafe_count_2000_na_price', 'cafe_count_2000_price_500', 'cafe_count_2000_price_1000', 'cafe_count_2000_price_1500', 'cafe_count_2000_price_2500', 'cafe_count_2000_price_4000', 'cafe_count_2000_price_high', 'big_church_count_2000', 'church_count_2000', 'leisure_count_2000', 'office_count_3000', 'office_sqm_3000', 'cafe_count_3000', 'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'cafe_count_3000_na_price', 'cafe_count_3000_price_500', 'cafe_count_3000_price_1000', 'cafe_count_3000_price_1500', 'cafe_count_3000_price_2500', 'cafe_count_3000_price_4000', 'cafe_count_3000_price_high', 'big_church_count_3000', 'church_count_3000', 'leisure_count_3000', 'office_count_5000', 'office_sqm_5000', 'cafe_count_5000', 'cafe_sum_5000_max_price_avg', 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high', 'big_church_count_5000', 'church_count_5000', 'leisure_count_5000', 'sport_count_5000']]

In [17]:
X_test = corr_train.drop(["id"], axis=1)
Y_test = train_joined['price_doc']

Дефолтный XGBRegressor

In [19]:
regr = XGBRegressor()
regr.fit(X_train, Y_train)
Y_pred = regr.predict(X_test)
metrics.mean_squared_error(Y_test, Y_pred)

13677586841609.531

Посмотрим разные параметры

In [20]:
xgb_reg = xgb.XGBRegressor()
params = {'max_depth': [5, 7, 9], 'colsample_bytree': [0.3, 0.5, 0.7], 'n_estimators': [50, 100, 200]}

xgb_cv = GridSearchCV(xgb_reg, params)
xgb_cv = xgb_cv.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [21]:
xgb_cv.best_params_

{'colsample_bytree': 0.3, 'max_depth': 9, 'n_estimators': 50}

In [25]:
regr = XGBRegressor(colsample_bytree=0.3, max_depth=9, n_estimators=50)
regr.fit(X_train, Y_train)
Y_pred = regr.predict(X_test)
metrics.mean_squared_error(Y_test, Y_pred)

7344887894653.413